## [reverse](https://docs.djangoproject.com/en/3.0/ref/urlresolvers/)
In your urls.py define this:
```python
url(r'^foo$', some_view, name='url_name'),
```
In a template you can then refer to this url as:
```html
<!-- django <= 1.4 -->
<a href="{% url url_name %}">link which calls some_view</a>

<!-- django >= 1.5 or with {% load url from future %} in your template -->
<a href="{% url 'url_name' %}">link which calls some_view</a>
```
This will be rendered as:
```html
<a href="/foo/">link which calls some_view</a>
```

Now say you want to do something similar in your views.py - e.g. you are handling some other url (not /foo/) in some other view (not some_view) and you want to redirect the user to /foo/ (often the case on successful form submission).

You could just do:
```python
return HttpResponseRedirect('/foo/')
```
But what if you want to change the url in future? You'd have to update your urls.py and all references to it in your code. This violates DRY (Don't Repeat Yourself), the whole idea of editing one place only, which is something to strive for.

Instead, you can say:
```python
from django.urls import reverse
return HttpResponseRedirect(reverse('url_name'))
```

## [resolve](https://docs.djangoproject.com/en/3.0/ref/urlresolvers/#resolve)
The resolve() function can be used for resolving URL paths to the corresponding view functions. 
If the URL does not resolve, the function raises a Resolver404 exception (a subclass of Http404) .

One possible use of resolve() would be to test whether a view would raise a Http404 error before redirecting to it:
```python
from urllib.parse import urlparse
from django.urls import resolve
from django.http import Http404, HttpResponseRedirect

def myview(request):
    next = request.META.get('HTTP_REFERER', None) or '/'
    response = HttpResponseRedirect(next)

    # modify the request and response as required, e.g. change locale
    # and set corresponding locale cookie

    view, args, kwargs = resolve(urlparse(next)[2])
    kwargs['request'] = request
    try:
        view(*args, **kwargs)
    except Http404:
        return HttpResponseRedirect('/')
    return response
 ```